In [67]:
import keras
import tensorflow as tf

In [68]:
#import keras backend for custom loss definition
import keras.backend as K

In [69]:
#layers import
from keras.layers import Input
from keras.layers import Conv1D, Conv2D, Dense, Flatten, Reshape, Dropout
from keras.layers import pooling
from keras.layers.merge import concatenate
from keras.layers import Activation

In [70]:
#optimizers
from keras.optimizers import adam

In [71]:
#initilizers
from keras.initializers import RandomNormal

In [72]:
# callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [73]:
#metrics 
from keras import metrics

In [74]:
#functional api model
from keras.models import Model

In [75]:
#pretrained models for a try
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

In [76]:
class Dpsh_model():
    def __init__(self, hash_length, regularization, cnn_model, include_top=True, input_shape=None):
        '''
        hash_length: the expected hash bit length. usually 24 or 48
        regularization: the term used in the loss in the paper. Seems 10-100 is good.
        cnn_model: the cnn_model instance used as feature extractor, can be ResNet50, VGG16, InceptionV3
        input_shape: the shape of input image like (224,224,3)
        '''
        self.cnn_model = cnn_model(include_top=include_top,input_shape=(224,224,3))
        
        # input layer
        self.input = Input(shape=input_shape, name='image')
        
        self.feature = self.cnn_model(self.input)
        
        # if the cnn model is only conv layers, we add one dense layer. and use reshape to make sure it is [batch size, ?]
        if include_top == False:
            dense_layer = Dense(4096, activation='relu')
            dropout = Dropout(0.3)
            self.feature = dropout(dense_layer(self.feature))
            #self.feature = dropout(dense_layer(self.feature))
            self.feature = Reshape((-1,))(self.feature)
        
        # hash function layer. 
        hash_layer = Dense(hash_length,kernel_initializer=RandomNormal(mean=0, stddev=0.01), name='hash_map')
        self.hash = hash_layer(self.feature)
       
       # loss func define
        def loss_func(labels,predicted):
            #labels is in (batch_size, n_class), predicted is (batch_size, bits)
            '''
            theta = tf.mul(1.0 / 2, tf.matmul(U0, tf.transpose(Ux)))
                B_code = tf.sign(U0)
            loss = tf.div(
                    (- 2.0 * tf.reduce_sum(tf.mul(S, theta) - (tf.max(0,theta)+tf.log(tf.exp(tf.abs(-theta))+1)))) + config.lamda * tf.reduce_sum(tf.pow((B_code - U0), 2)),
                    float(config.N_size*config.batch_size))
            '''
            u = K.sign(predicted)
            theta = K.dot(u, K.transpose(u))/2
            sim = K.dot(labels, K.transpose(labels))
            loss = -K.mean(sim*theta - K.log(1+ K.exp(theta))) + \
                    regularization * K.mean(K.pow((u-predicted),2))
            
            return loss
        
        # we also need custom the metrics.Here we use loss as metrics
        def metrics_func(labels,predicted):
            return loss_func(labels, predicted)
        
        # construct the mode.
        self.dpsh_model = Model(inputs=self.input, outputs=self.hash)
        
        #compile with custom loss
        self.dpsh_model.compile(optimizer='adam', loss=loss_func, metrics=[metrics_func])

In [78]:
#Below are hyper parameters.
hash_length = 48
regularization = 60 
cnn_model = ResNet50
include_top=False 
input_shape=(224,224,3)

dpsh = Dpsh_model(hash_length, regularization, cnn_model, include_top, input_shape)

In [82]:
dpsh.dpsh_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1, 4096)        8392704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 4096)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 4096)              0         
_________________________________________________________________
hash_map (Dense)             (None, 48)                196656    
Total params: 32,177,072.0
Trainable params: 32,123,952.0
Non-trainable params: 53,120.0
_____________________________________________________

In [30]:
%run data_process_new.ipynb

In [31]:
data, label = get_input_and_labels()

In [34]:
print(data.shape, label.shape)

(5000, 224, 224, 3) (5000, 10)


In [32]:
val_data, val_label = get_input_and_labels(train=False)

In [80]:
early_stoping = EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='min')
filepath = 'weights_new_wotop.{epoch:02d}-{val_loss:.2f}.hdf5'
model_ckp = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', period=1)

In [ ]:
hist = dpsh.dpsh_model.fit(data, label, validation_data=(val_data, val_label), epochs=15, batch_size=128, verbose=2, shuffle=False, callbacks=[early_stoping,model_ckp])

Train on 5000 samples, validate on 1000 samples
Epoch 1/15
3532s - loss: 24.3166 - metrics_func: 24.3166 - val_loss: 44.1357 - val_metrics_func: 44.1357
Epoch 2/15
3513s - loss: 20.9510 - metrics_func: 20.9510 - val_loss: 22.8627 - val_metrics_func: 22.8627
Epoch 3/15


In [54]:
predicted = dpsh.dpsh_model.predict(val_data[0:20])

In [55]:
hash_c = np.sign(predicted)

In [56]:
print(val_label[0:20])

[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


In [58]:
print(hash_c[3], )
print(hash_c[5], )

[-1. -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.
  1.  1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.]
[-1. -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.
  1.  1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.]


In [59]:
print(hash_c[0], )

[-1. -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.
  1.  1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.]


In [60]:
print(hash_c[19], )

[-1. -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1.  1. -1.  1. -1.
  1.  1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1. -1.]


In [64]:
hash_c[6] == hash_c[9]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True], dtype=bool)